## Get frames from a given class (car)

This script aims to retrieve class samples (car, people, etc.) from event frames stored at DynamoDB and KVS.

In [ ]:
from datetime import datetime, timedelta, timezone
import random
import sys
from collections import defaultdict
from tqdm import tqdm
from glob import glob
import os

sys.path.append('..')
from event_query_app.event_query import EventQuery

CLASS = "car"
DEVICE_ID = "B8A44FB3A1F9__front"

THRESHOLD = 0.5
CAP_PER_DAY_PER_DEVICE = 20
DAYS_BACK = 30

event_query = EventQuery()
END_DATE = datetime.now(timezone.utc)

from kvs_event_consumer import KVSEventConsumer
import numpy as np
import cv2

kvs_consumer = KVSEventConsumer()
for day_offset in range(DAYS_BACK):
    day_start = END_DATE - timedelta(days=day_offset + 1)
    day_end = END_DATE - timedelta(days=day_offset)
    print(f"Processing events from {day_start} to {day_end}")

    daily_results = event_query.query_events(
        start_date=day_start,
        end_date=day_end,
        target_classes=[CLASS],
        threshold=THRESHOLD,
        device_ids=[DEVICE_ID]
    )

    sampled_daily_events = random.sample(daily_results, CAP_PER_DAY_PER_DEVICE)
    for event in tqdm(sampled_daily_events):
        existing_files = glob(os.path.join("B8A44FB3A1F9__front_cars", f"event_{event['timestamp']}_frame_*.png"))
        if existing_files:
            continue  # Skip if already processed


        frames = kvs_consumer.get_event_frames(event)
        indices = np.linspace(0, len(frames) - 1, num=3, dtype=int)
        for i in indices:
            frame_bgr = cv2.cvtColor(frames[i], cv2.COLOR_RGB2BGR)
            cv2.imwrite(f"B8A44FB3A1F9__front_cars/event_{event['timestamp']}_frame_{i}.png", frame_bgr)
            
        

WHATS is better here: 

(1) - get the event from KVS and run a detector on it saving nice frames.

(2) - try to already use detections made before and sync with KVS

Now, for every image in the dataset, tries to run ALPR on it